In [ ]:
cd /notebooks/notebooks/ER_HDM/Address_POC

In [ ]:
import pandas as pd
import numpy as np
# import difflib
import pyodbc
import urllib
# from difflib import SequenceMatcher
import sqlalchemy
from collections import OrderedDict 
# from swifter import swifter
# import timeit
import datetime

In [ ]:
from configparser import ConfigParser
config = ConfigParser()
config.read('config.ini')
config_data=config['SERVER']
Key=config_data["read"]
print(Key)

In [ ]:
#Establishing connection
conn=pyodbc.connect(Key)
cursor=conn.cursor()

In [ ]:
#Code to Generate Alpha Num Strings
def Alpha_Num_Strings(Add_Str):
    Token_Dict={
        '1':['1st' ,'first','i'],
'2':['2nd','second','ii'],
'3':['3rd','third','iii'],
'4':['4th','fourth','iv'],
'5':['5th','fifth','v'],
'6':['6th','sixth','vi'],
'7':['7th','seventh','vii'],
'8':['8th','eighth','viii'],
'9':['9th','ninth','ix'],
'10':['10th','tenth','x'],
'11':['11th','eleventh','xi'],
'12':['12th','twelfth','xii'],
'13':['13th','thirteenth','xiii'],
'14':['14th','fourteenth','xiv'],
'15':['15th','fifteenth','xv'],
'16':['16th','sixteenth','xvi'],
'17':['17th','seventeenth','xvii'],
'18':['18th','eighteenth','xviii'],
'19':['19th','nineteenth','xix'],
'20':['20th','twentieth','xx'],
'21':['21st'],
'22':['22nd'],
'23':['23rd'],
'24':['24th'],
'25':['25th'],
'26':['26th'],
'27':['27th'],
'28':['28th'],
'29':['29th'],
'30':['30th'],
'31':['31st'],
'32':['32nd'],
'33':['33rd'],
'34':['34th'],
'35':['35th'],
'36':['36th'],
'37':['37th'],
'38':['38th'],
'39':['39th'],
'40':['40th'],
'41':['41st'],
'42':['42nd'],
'43':['43rd'],
'44':['44th'],
'45':['45th'],
'46':['46th'],
'47':['47th'],
'48':['48th'],
'49':['49th'],
'50':['50th'],
'road':['gali','rd','st','street'],
'floor':['manjil','mala','manzil'],
'number':['no','no.','number-','number:'],
'housenumber':['h.no.','hno','hno:','hno-'],
'':['address','address.','address-','address:','residential','residential.','residential-','residential:','vill','vill.','vill-','vill:','village','village.','village-','village:','post','post.','post-','post:','dist','dist.','dist-','dist:','district','district.','district-','district:'],
'doornumber':['dno','dno-','dno:','d.no.'],
'plot':['khasra'],
'upperground':['ug'],
'lowerground':['lg']

    }
    
        
    
    Fiu_Add2=list((Add_Str).lower().replace(',',' ').split())
    #print(Fiu_Add)
    
    Fiu_Add=[]
    for i in Fiu_Add2:
        if i not in Fiu_Add:
            Fiu_Add.append(i)
    
    Formatted_Fiu_Address=[]
#     if str(Fiu_Pincode) in Fiu_Add:
#         Fiu_Add.remove(str(Fiu_Pincode))
    for i in list(Fiu_Add):
        flag=0
        for k,v in Token_Dict.items():
            if i in v:
                Formatted_Fiu_Address.append(k)
                flag=1
                break
            
        if flag==0:
            Formatted_Fiu_Address.append(i)
    if '' in Formatted_Fiu_Address:
        Formatted_Fiu_Address.remove('')
    Fiu_Num=[]
    Fiu_Alpha=[]
    for i in Formatted_Fiu_Address:
        if i.isalpha():
            Fiu_Alpha.append(i)
        else:
            Fiu_Num.append(i)

#     print ('Fiu_Num',Fiu_Num,end='\n\n')
#     print ('Fiu_Alpha',Fiu_Alpha,end='\n\n')
    
    Fiu_Num_String=" ".join(Fiu_Num)
    Fiu_Alpha_String=" ".join(Fiu_Alpha)
    
#     print('Alpha_Num_Strings called, output: ',Fiu_Alpha_String,Fiu_Num_String)
    
    Fiu_Add_Tokens=(Add_Str).lower().replace(',',' ').replace('"',' ').replace('\'',' ').split()
    Clean_Input_Address_List=[]
    for i in Fiu_Add_Tokens:
        if i not in Clean_Input_Address_List:
            flag=0
            for k,v in Token_Dict.items():
                if i in v:
                    Clean_Input_Address_List.append(k)
                    flag=1
                    break
            
            if flag==0:
                Clean_Input_Address_List.append(i)
                
    if '' in Formatted_Fiu_Address:
        Clean_Input_Address_List.remove('')
#     print(Clean_Input_Address_List)
    Clean_Input_Address=(' '.join(Clean_Input_Address_List))
#     print(Clean_Input_Address)
#     type(Fiu_Alpha_String)

#     return Fiu_Alpha_String 
#     print()
    return(Fiu_Alpha_String,Fiu_Num_String,Clean_Input_Address)
#     ADDR_OUT=list(Fiu_Alpha_String,Fiu_Num_String,Clean_Input_Address)
#     print(ADDR_OUT)
print('Job Started :' , datetime.datetime.now())

# SOURCE
ADDR_Stg = pd.read_sql("""
        SELECT 
       [Fiu_Batch_Id] 
	  ,[Report_Id]
	  ,[Batch_Id]
	  ,[fiu_entity_type]
	  ,[fiu_fingate_person_entity_id]
      ,[source]
      ,[rec_type]
      ,CASE WHEN FIU_PINCODE IS NOT NULL THEN REPLACE([fiu_AddressLine1],fiu_pincode,'') ELSE [fiu_AddressLine1] END  AS Fiu_AL1
      ,[fiu_country] as Fiu_Country
      ,[fiu_pincode]
      ,[fiu_locality] as Fiu_Locality
      ,[fiu_cityvillagetown] as Fiu_City
      ,[fiu_state] as Fiu_State
      ,[fiu_district]  as Fiu_District
      FROM
     Fincore_Staging.hdm_Unresolved_Address_Temp_Pre_perftest
     where is_valid_address=1
""", conn)
print('Data loaded in Dataframe from Pre Stage :' , datetime.datetime.now())
ADDR_Stg['Fiu_City']=ADDR_Stg['Fiu_City'].astype(str)
ADDR_Stg['Final_Add']=np.where(ADDR_Stg['Fiu_AL1']!='None',ADDR_Stg['Fiu_AL1'].apply(str),'')
ADDR_Stg['Final_Add']=np.where(ADDR_Stg['Fiu_Locality']!='None',(ADDR_Stg['Final_Add'].apply(str)+' '+ADDR_Stg['Fiu_Locality'].apply(str)),ADDR_Stg['Final_Add'])
ADDR_Stg['Final_Add']=np.where(ADDR_Stg['Fiu_City']!='None',(ADDR_Stg['Final_Add'].apply(str)+' '+ADDR_Stg['Fiu_City'].apply(str)),ADDR_Stg['Final_Add'])
ADDR_Stg['Final_Add']=np.where(ADDR_Stg['Fiu_District']!='None',(ADDR_Stg['Final_Add'].apply(str)+' '+ADDR_Stg['Fiu_District'].apply(str)),ADDR_Stg['Final_Add'])
ADDR_Stg['Final_Add']=np.where(ADDR_Stg['Fiu_State']!='None',(ADDR_Stg['Final_Add'].apply(str)+' '+ADDR_Stg['Fiu_State'].apply(str)),ADDR_Stg['Final_Add'])
ADDR_Stg['Final_Add']=np.where(ADDR_Stg['Fiu_Country']!='None',(ADDR_Stg['Final_Add'].apply(str)+' '+ADDR_Stg['Fiu_Country'].apply(str)),ADDR_Stg['Final_Add'])


ADDR_Stg2 = ADDR_Stg


print('Function Call Started :' , datetime.datetime.now())
ADDR_Stg2['Alpha_Num_OFA']= (ADDR_Stg['Final_Add'].apply(Alpha_Num_Strings))
ADDR_Stg2['Alpha_String']=ADDR_Stg2['Alpha_Num_OFA'].apply(lambda x : x[0])
ADDR_Stg2['Num_String']=ADDR_Stg2['Alpha_Num_OFA'].apply(lambda x : x[1])
ADDR_Stg2['Original_Full_Address']=ADDR_Stg2['Alpha_Num_OFA'].apply(lambda x : x[2])
ADDR_Stg2['Final_Add']='Final_Add'
ADDR_Stg2['Alpha_Num_OFA']='Alpha_Num_OFA'
# SOURCE
quoted = urllib.parse.quote_plus(Key)
print('Function Call Completed :' , datetime.datetime.now())
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
# (ADDR_Stg2).to_sql('HDM_Unresolved_Address_Temp_Alpha_Num_Perftest_Optimized', schema='Fincore_Staging', con = engine, chunksize=100, method='multi', index=False, if_exists='replace')


cursor.execute('''Truncate table Fincore_Staging.HDM_Unresolved_Address_Temp_Alpha_Num_Perftest_Optimized''')
conn.commit()

print('HDM_Unresolved_Address_Temp_Alpha_Num_Perftest_Optimized Table Truncated :' , datetime.datetime.now())

rows_as_json_Chunk1 = ADDR_Stg2[0:1000000].to_json(orient='records')
rows_as_json_Chunk2 = ADDR_Stg2[1000000:2000000].to_json(orient='records')
rows_as_json_Chunk3 = ADDR_Stg2[2000000:3000000].to_json(orient='records')
rows_as_json_Chunk4 = ADDR_Stg2[3000000:4000000].to_json(orient='records')
rows_as_json_Chunk5 = ADDR_Stg2[4000000:].to_json(orient='records')

print('Insert Started :' , datetime.datetime.now())
sql = f'''
INSERT INTO Fincore_Staging.HDM_Unresolved_Address_Temp_Alpha_Num_Perftest_Optimized
(Fiu_Batch_Id
,Report_Id
,Batch_Id
,fiu_entity_type
,fiu_fingate_person_entity_id
,source
,rec_type
,Fiu_AL1
,Fiu_Country
,fiu_pincode
,Fiu_Locality
,Fiu_City
,Fiu_State
,Fiu_District
,Final_Add
,Alpha_Num_OFA
,Alpha_String
,Num_String
,Original_Full_Address
)
SELECT 
Fiu_Batch_Id
,Report_Id
,Batch_Id
,fiu_entity_type
,fiu_fingate_person_entity_id
,source
,rec_type
,Fiu_AL1
,Fiu_Country
,fiu_pincode
,Fiu_Locality
,Fiu_City
,Fiu_State
,Fiu_District
,Final_Add
,Alpha_Num_OFA
,Alpha_String
,Num_String
,Original_Full_Address
FROM
    OPENJSON(?)
    WITH (
        [First Name] nvarchar(50) '$."First Name"'
        ,Fiu_Batch_Id	bigint  '$."Fiu_Batch_Id"'
        ,Report_Id	bigint '$."Report_Id"'
        ,Batch_Id	bigint '$."Batch_Id"'
        ,fiu_entity_type	varchar(max) '$."fiu_entity_type"'
        ,fiu_fingate_person_entity_id	bigint '$."fiu_fingate_person_entity_id"'
        ,source	bigint '$."source"'
        ,rec_type	varchar(max) '$."rec_type"'
        ,Fiu_AL1	varchar(max) '$."Fiu_AL1"'
        ,Fiu_Country	varchar(max) '$."Fiu_Country"'
        ,fiu_pincode	varchar(max) '$."fiu_pincode"'
        ,Fiu_Locality	varchar(max) '$."Fiu_Locality"'
        ,Fiu_City	varchar(max) '$."Fiu_City"'
        ,Fiu_State	varchar(max) '$."Fiu_State"'
        ,Fiu_District	varchar(max) '$."Fiu_District"'
        ,Final_Add	varchar(max) '$."Final_Add"'
        ,Alpha_Num_OFA	varchar(max) '$."Alpha_Num_OFA"'
        ,Alpha_String	varchar(max) '$."Alpha_String"'
        ,Num_String	varchar(max) '$."Num_String"'
        ,Original_Full_Address	varchar(max) '$."Original_Full_Address"'
    )
'''

cursor = engine.raw_connection().cursor()
cursor.execute(sql, rows_as_json_Chunk1)
cursor.execute(sql, rows_as_json_Chunk2)
cursor.execute(sql, rows_as_json_Chunk3)
cursor.execute(sql, rows_as_json_Chunk4)
cursor.execute(sql, rows_as_json_Chunk5)
cursor.commit()
print('Job Completed :' , datetime.datetime.now())

In [ ]:

# import threading
 
# def parallel_insert(df1):
# #     config = ConfigParser()
# #     config.read(r'config.ini')
# #     connection_string = config.get('SERVER', 'connection')
# #     conn = pyodbc.connect(connection_string)
# #     cursor = conn.cursor()
 
# #     connection_url = sa.engine.URL.create(
# #         "mssql+pyodbc", 
# #         query=dict(odbc_connect=connection_string)
# #     )
#     quoted = urllib.parse.quote_plus(Key)
#     engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted), fast_executemany=True)
# #     engine = sa.create_engine(connection_url, fast_executemany=True)
#     df1.to_sql("HDM_Unresolved_Address_Temp_Alpha_Num_Perftest_Optimized", engine, schema="Fincore_Staging", if_exists="replace", index=False,chunksize=10000)
 
# # temp1 = ADDR_Stg2[0:250000]
# # temp2 = ADDR_Stg2[250000:500000] 
# # temp3 = ADDR_Stg2[500000:750000] 
# # temp4 = ADDR_Stg2[750000:] 

# # temp1 = ADDR_Stg2[0:1000]
# # temp2 = ADDR_Stg2[1000:2000] 
# # temp3 = ADDR_Stg2[2000:3000] 
# temp4 = ADDR_Stg2[0:] 


# # one=threading.Thread(target=parallel_insert,args=(temp1,))
# # two=threading.Thread(target=parallel_insert,args=(temp2,))
# # three=threading.Thread(target=parallel_insert,args=(temp3,))
# four=threading.Thread(target=parallel_insert,args=(temp4,))
 
# # one.start()
# # two.start()
# # three.start()
# four.start()
 
# # one.join()
# # two.join()
# # three.join()
# four.join()
# print('Job completed :' , datetime.datetime.now())
